In [27]:
import os
from os import path
import numpy as np
import pandas as pd
import pymc3 as mc
import theano
import seaborn as sns

np.random.seed(123)
%matplotlib inline


In [13]:
import pymc3.examples.data as data_dir
data_dir = data_dir.__path__._path[0]
df = pd.read_csv(path.join(data_dir, "radon.csv"))
df.shape

(919, 30)

In [14]:
df.head()

,Unnamed: 0,idnum,state,state2,stfips,zip,region,typebldg,floor,room,...,pcterr,adjwt,dupflag,zipflag,cntyfips,county,fips,Uppm,county_code,log_radon
0,0,5081.0,MN,MN,27.0,55735,5.0,1.0,1.0,3.0,...,9.7,1146.499190,1.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.832909
1,1,5082.0,MN,MN,27.0,55748,5.0,1.0,0.0,4.0,...,14.5,471.366223,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.832909
2,2,5083.0,MN,MN,27.0,55748,5.0,1.0,0.0,4.0,...,9.6,433.316718,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,1.098612
3,3,5084.0,MN,MN,27.0,56469,5.0,1.0,0.0,4.0,...,24.3,461.623670,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.095310
4,4,5085.0,MN,MN,27.0,55011,3.0,1.0,0.0,4.0,...,13.8,433.316718,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,1.163151


In [29]:
county_names = df.county.unique()
county_idx = df.county_code.values
n_counties = len(county_names)
df['log_radon'] = df['log_radon'].astype(theano.config.floatX)
X = df[['county', 'log_radon', 'floor']]

In [23]:
traces = dict()
for county_name in county_names:
    # get data for the current county
    X_one = X.loc[X.county == county_name].reset_index(drop=True)
    
    log_radon = X_one.log_radon
    floor = X_one.floor.values
    
    with pm.Model() as one_model:
        # intercept prior
        intercept = pm.Normal('intercept', mu=0, sd=1)
        # slope prior
        slope = pm.Normal('slope', mu=0, sd=1)
        # model error prior
        eps = pm.HalfCauchy('eps', beta=1)
        
        # define the model
        model_est = intercept + slope * floor
        
        # define the data likelihood
        y_lik = pm.Normal('y_lik', mu=model_est, sd=eps, observed=log_radon)
        
        # sample with the model
        trace = pm.sample(progressbar=True, n_init=100, draws=100)
        
    traces[county_name] = trace

Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, slope, intercept]
Sampling 4 chains: 100%|██████████| 2400/2400 [00:00<00:00, 3969.34draws/s]
The acceptance probability does not match the target. It is 0.886691850698103, but should be close to 0.8. Try to increase the number of tuning steps.
Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, slope, intercept]
Sampling 4 chains: 100%|██████████| 2400/2400 [00:00<00:00, 4188.49draws/s]
The acceptance probability does not match the target. It is 0.879785382540666, but should be close to 0.8. Try to increase the number of tuning steps.
Only 100 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, slope, intercept]
Sampling 4 chains:

In [25]:
with pm.Model() as hierarchical_model:
    # hyper-priors
    mu_alpha = pm.Normal('mu_alpha', mu=0, sd=1)
    mu_beta = pm.Normal('mu_beta', mu=0, sd=1)
    sigma_alpha = pm.HalfCauchy('sigma_alpha', beta=1)
    sigma_beta = pm.HalfCauchy('sigma_beta', beta=1)
    
    # the intercepts (alphas) for each county
    alphas = pm.Normal('county_alphas', mu=mu_alpha, sd=sigma_alpha, shape=len(county_names))
    # the slopes (betas) for each county
    betas = pm.Normal('county_betas', mu=mu_beta, sd=sigma_beta, shape=len(county_names))
    
    # the hierarchical model
    radon_est = alphas[df.county_code.values] + betas[df.county_code.values] * df.floor.values
    
    # the model error
    eps = pm.HalfCauchy('eps', beta=1)
    
    # define the data likelihood under this model
    y_lik = pm.Normal('y_lik', mu=radon_est, sd=eps, observed=df.log_radon.values)

with hierarchical_model:
    hierarchical_trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, county_betas, county_alphas, sigma_beta, sigma_alpha, mu_beta, mu_alpha]
Sampling 4 chains: 100%|██████████| 4000/4000 [00:08<00:00, 465.31draws/s]
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.6903845264606879, but should be close to 0.8. Try to increase the number of tuning steps.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9554717005910119, but should be close to 0.8. Try to increase the number of tuning steps.
There were 164 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.49970794249328465, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance pr